In [1]:
%load_ext autoreload
%autoreload 2

from dependencies import *

Loading dependencies we have already seen...
Importing ray...
Done...


In [2]:
import h5py

def setup(filepath):
    with h5py.File(filepath, 'r') as f:
        X_train = f["train_x"][:]
        y_train = f["train_y"][:]
        group_train = f["train_groups"][:]
        train_wells = f["train_groups"].attrs["well_names"]        
        
        X_test = f["test_x"][:]
        y_test = None
#         y_test = f["test_y"]
        group_test = f["test_groups"][:]
        test_wells = f["test_groups"].attrs["well_names"]

    return X_train, y_train, group_train, X_test, y_test, group_test, (train_wells, test_wells)

X_train, y_train, group_train, X_test, y_test, group_test, well_names = setup('../datasets/seg_2016_facies/la_team_5_data.h5py')

In [3]:
# XGBClassifier(learning_rate=0.12,
#               max_depth=3,
#               min_child_weight=10,
#               n_estimators=150,
#               seed=seed,
#               colsample_bytree=0.9)

In [4]:
from os import path
filepath = path.abspath('../datasets/seg_2016_facies/la_team_5_data.h5py')
print(filepath)

/Users/stevejpurves/dev/swung/ray_playground/datasets/seg_2016_facies/la_team_5_data.h5py


In [5]:
from sklearn.model_selection import LeavePGroupsOut
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from scipy.signal import medfilt
from filelock import FileLock

def e2e_train_and_test(config, **kwargs):
    
    #threadsafe
#     filepath = path.abspath('../datasets/seg_2016_facies/la_team_5_data.h5py')
    with FileLock("./data.lock"):
        X, y, groups, X_test, y_test, group_test, well_names = setup(kwargs['filepath'])
    
    # chose your CV strategy
    splitter = LeavePGroupsOut(1)
    
    # run k fold training and validation
    f1_scores = [] # keep hold of all individual scores
    for train_ind, val_ind in splitter.split(X, y, groups=groups):
        pipeline = make_pipeline(RobustScaler(),
                                  XGBClassifier())

        pipeline.set_params(**config)
        pipeline.fit(X[train_ind], y[train_ind])
        
        y_pred = pipeline.predict(X[val_ind])
        
        f1_scores.append(f1_score(y_pred, y[val_ind], average='micro'))
    
        # Clean isolated facies for each well
        y_pred = medfilt(y_pred, kernel_size=5)
    
    # use tunes reporter
    tune.track.log(mean_f1_score=np.array(f1_scores).mean(),
                std_f1_score=np.array(f1_scores).std(),
                # and we can actually add any metrics we like
                done=True)

In [6]:
tuning_config = {
    'xgbclassifier__learning_rate': 0.12,
    'xgbclassifier__max_depth': 3,
    'xbgclassifier__min_child_weight' :10,
    'xbgclassifier__n_estimators': 150,
    'xgbclassifier__seed':1773,
    'xgbclassifier__colsample_bytree':0.9
}

In [7]:
ray_tuning_config = {
    'xgbclassifier__learning_rate': tune.loguniform(0.001, 0.5),
    'xgbclassifier__max_depth': tune.randint(1, 10),
    'xgbclassifier__min_child_weight': tune.loguniform(0.1,100),
    'xgbclassifier__n_estimators': tune.randint(5,200),
    'xgbclassifier__colsample_bytree': tune.choice([0.4, 0.6, 0.8, 1.0]),
    'xgbclassifier__lambda': tune.choice([0,1]),
    'xgbclassifier__seed': 42
}

In [14]:
ray.shutdown()
# ray.init(num_cpus=6, num_gpus=0, include_webui=True)

ray.init(address='auto', redis_password='5241590000000000')

2020-06-09 12:43:20,160	WARNING worker.py:808 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.


{'node_ip_address': '192.168.1.36',
 'raylet_ip_address': '192.168.1.36',
 'redis_address': '192.168.1.36:62769',
 'object_store_address': '/tmp/ray/session_2020-06-08_16-39-31_048190_82855/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-08_16-39-31_048190_82855/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-06-08_16-39-31_048190_82855'}

In [17]:
filepath = path.abspath('../datasets/seg_2016_facies/la_team_5_data.h5py')

def e2e_seg(config):
    return e2e_train_and_test(config, filepath=filepath)

analysis = tune.run(
                e2e_seg,
                config=ray_tuning_config,
                num_samples=10, # Specify the number of samples to make from (non grid) distributions
                resources_per_trial=dict(cpu=1, gpu=0),
                local_dir="~/ray_results/seg_facies")

Trial name,status,loc,xgbclassifier__colsample_bytree,xgbclassifier__lambda,xgbclassifier__learning_rate,xgbclassifier__max_depth,xgbclassifier__min_child_weight,xgbclassifier__n_estimators
e2e_seg_00000,RUNNING,,0.4,0,0.00106545,2,7.38127,48
e2e_seg_00001,PENDING,,0.8,0,0.06624,1,2.31399,35
e2e_seg_00002,PENDING,,0.6,0,0.0774589,1,1.99245,159
e2e_seg_00003,PENDING,,0.8,1,0.020335,1,0.498322,123
e2e_seg_00004,PENDING,,0.8,0,0.049454,8,5.84613,184
e2e_seg_00005,PENDING,,0.4,0,0.032176,8,0.114637,11
e2e_seg_00006,PENDING,,0.6,1,0.0134216,2,0.22047,97
e2e_seg_00007,PENDING,,0.8,1,0.0106233,1,35.9295,134
e2e_seg_00008,PENDING,,1,0,0.00129878,4,2.07083,143
e2e_seg_00009,PENDING,,0.8,1,0.0494578,4,14.3154,13


(pid=39089) 2020-06-09 12:44:49,037	INFO trainable.py:217 -- Getting current IP.
(pid=39179) 2020-06-09 12:44:49,611	INFO trainable.py:217 -- Getting current IP.


2020-06-09 12:44:54,610	INFO logger.py:269 -- Removed the following hyperparameter values when logging to tensorboard: {'xgbclassifier__lambda': 0}


Result for e2e_seg_00001:
  date: 2020-06-09_12-44-54
  done: true
  experiment_id: 04e257aa378d4368aa3fa89037c18b9b
  experiment_tag: 1_xgbclassifier__colsample_bytree=0.8,xgbclassifier__lambda=0,xgbclassifier__learning_rate=0.06624,xgbclassifier__max_depth=1,xgbclassifier__min_child_weight=2.314,xgbclassifier__n_estimators=35
  hostname: Steves-MacBook-Pro.local
  iterations_since_restore: 1
  mean_f1_score: 0.44021228454009326
  node_ip: 192.168.1.36
  pid: 39179
  std_f1_score: 0.1633947419220141
  time_since_restore: 4.993320941925049
  time_this_iter_s: 4.993320941925049
  time_total_s: 4.993320941925049
  timestamp: 1591703094
  timesteps_since_restore: 0
  training_iteration: 0
  trial_id: '00001'
  


Trial name,status,loc,xgbclassifier__colsample_bytree,xgbclassifier__lambda,xgbclassifier__learning_rate,xgbclassifier__max_depth,xgbclassifier__min_child_weight,xgbclassifier__n_estimators,iter,total time (s)
e2e_seg_00000,RUNNING,,0.4,0,0.00106545,2,7.38127,48,,
e2e_seg_00001,TERMINATED,,0.8,0,0.06624,1,2.31399,35,0,4.99332
e2e_seg_00002,PENDING,,0.6,0,0.0774589,1,1.99245,159,,
e2e_seg_00003,PENDING,,0.8,1,0.020335,1,0.498322,123,,
e2e_seg_00004,PENDING,,0.8,0,0.049454,8,5.84613,184,,
e2e_seg_00005,PENDING,,0.4,0,0.032176,8,0.114637,11,,
e2e_seg_00006,PENDING,,0.6,1,0.0134216,2,0.22047,97,,
e2e_seg_00007,PENDING,,0.8,1,0.0106233,1,35.9295,134,,
e2e_seg_00008,PENDING,,1,0,0.00129878,4,2.07083,143,,
e2e_seg_00009,PENDING,,0.8,1,0.0494578,4,14.3154,13,,


...

Trial name,status,loc,xgbclassifier__colsample_bytree,xgbclassifier__lambda,xgbclassifier__learning_rate,xgbclassifier__max_depth,xgbclassifier__min_child_weight,xgbclassifier__n_estimators,iter,total time (s)
e2e_seg_00000,TERMINATED,,0.4,0,0.00106545,2,7.38127,48,0,7.58806
e2e_seg_00001,TERMINATED,,0.8,0,0.06624,1,2.31399,35,0,4.99332
e2e_seg_00002,TERMINATED,,0.6,0,0.0774589,1,1.99245,159,0,18.1751
e2e_seg_00003,TERMINATED,,0.8,1,0.020335,1,0.498322,123,0,17.2272
e2e_seg_00004,RUNNING,,0.8,0,0.049454,8,5.84613,184,,
e2e_seg_00005,TERMINATED,,0.4,0,0.032176,8,0.114637,11,0,6.82706
e2e_seg_00006,TERMINATED,,0.6,1,0.0134216,2,0.22047,97,0,19.2709
e2e_seg_00007,TERMINATED,,0.8,1,0.0106233,1,35.9295,134,0,18.3217
e2e_seg_00008,TERMINATED,,1,0,0.00129878,4,2.07083,143,0,89.1498
e2e_seg_00009,TERMINATED,,0.8,1,0.0494578,4,14.3154,13,0,6.28663


2020-06-09 12:47:55,737	INFO logger.py:269 -- Removed the following hyperparameter values when logging to tensorboard: {'xgbclassifier__lambda': 0}


Result for e2e_seg_00004:
  date: 2020-06-09_12-47-55
  done: true
  experiment_id: 8dac15294395471c801edf318c93ae2e
  experiment_tag: 4_xgbclassifier__colsample_bytree=0.8,xgbclassifier__lambda=0,xgbclassifier__learning_rate=0.049454,xgbclassifier__max_depth=8,xgbclassifier__min_child_weight=5.8461,xgbclassifier__n_estimators=184
  hostname: Steves-MacBook-Pro.local
  iterations_since_restore: 1
  mean_f1_score: 0.5758615002991206
  node_ip: 192.168.1.36
  pid: 39206
  std_f1_score: 0.08616120832349243
  time_since_restore: 159.9369261264801
  time_this_iter_s: 159.9369261264801
  time_total_s: 159.9369261264801
  timestamp: 1591703275
  timesteps_since_restore: 0
  training_iteration: 0
  trial_id: '00004'
  


Trial name,status,loc,xgbclassifier__colsample_bytree,xgbclassifier__lambda,xgbclassifier__learning_rate,xgbclassifier__max_depth,xgbclassifier__min_child_weight,xgbclassifier__n_estimators,iter,total time (s)
e2e_seg_00000,TERMINATED,,0.4,0,0.00106545,2,7.38127,48,0,7.58806
e2e_seg_00001,TERMINATED,,0.8,0,0.06624,1,2.31399,35,0,4.99332
e2e_seg_00002,TERMINATED,,0.6,0,0.0774589,1,1.99245,159,0,18.1751
e2e_seg_00003,TERMINATED,,0.8,1,0.020335,1,0.498322,123,0,17.2272
e2e_seg_00004,TERMINATED,,0.8,0,0.049454,8,5.84613,184,0,159.937
e2e_seg_00005,TERMINATED,,0.4,0,0.032176,8,0.114637,11,0,6.82706
e2e_seg_00006,TERMINATED,,0.6,1,0.0134216,2,0.22047,97,0,19.2709
e2e_seg_00007,TERMINATED,,0.8,1,0.0106233,1,35.9295,134,0,18.3217
e2e_seg_00008,TERMINATED,,1,0,0.00129878,4,2.07083,143,0,89.1498
e2e_seg_00009,TERMINATED,,0.8,1,0.0494578,4,14.3154,13,0,6.28663


Trial name,status,loc,xgbclassifier__colsample_bytree,xgbclassifier__lambda,xgbclassifier__learning_rate,xgbclassifier__max_depth,xgbclassifier__min_child_weight,xgbclassifier__n_estimators,iter,total time (s)
e2e_seg_00000,TERMINATED,,0.4,0,0.00106545,2,7.38127,48,0,7.58806
e2e_seg_00001,TERMINATED,,0.8,0,0.06624,1,2.31399,35,0,4.99332
e2e_seg_00002,TERMINATED,,0.6,0,0.0774589,1,1.99245,159,0,18.1751
e2e_seg_00003,TERMINATED,,0.8,1,0.020335,1,0.498322,123,0,17.2272
e2e_seg_00004,TERMINATED,,0.8,0,0.049454,8,5.84613,184,0,159.937
e2e_seg_00005,TERMINATED,,0.4,0,0.032176,8,0.114637,11,0,6.82706
e2e_seg_00006,TERMINATED,,0.6,1,0.0134216,2,0.22047,97,0,19.2709
e2e_seg_00007,TERMINATED,,0.8,1,0.0106233,1,35.9295,134,0,18.3217
e2e_seg_00008,TERMINATED,,1,0,0.00129878,4,2.07083,143,0,89.1498
e2e_seg_00009,TERMINATED,,0.8,1,0.0494578,4,14.3154,13,0,6.28663


In [16]:
print("Best config: ", analysis.get_best_config(metric="mean_f1_score"))

Best config:  {'xgbclassifier__learning_rate': 0.04861679346423644, 'xgbclassifier__max_depth': 6, 'xgbclassifier__min_child_weight': 19.96980580643878, 'xgbclassifier__n_estimators': 162, 'xgbclassifier__colsample_bytree': 0.4, 'xgbclassifier__lambda': 1, 'xgbclassifier__seed': 42}


In [ ]:
df = analysis.dataframe()
top_n_df = df.nlargest(10, "mean_f1_score")

In [ ]:
top_n_df.head()

In [ ]:
plot_some_tune_results(top_n_df, (0.5, 1.0))

In [ ]:
%load_ext tensorboard
from tensorboard import notebook 
%tensorboard --logdir "~/ray_results/seg_facies"
notebook.display(height=1000)

In [ ]:
ray.shutdown()